# Creates lists of quantitative and qualitative variables.


<font color = "red">  >>> In this notebook, the list of qualitative and quantitative variables are obtained with the first dataset containing the 13 time series. <<<

In [5]:
# Librairies pour se connecter au drive: 
from google.colab import drive
import os

# Connexion au drive:
drive.mount('/content/drive') 

# Chemin du dossier contenant le projet: 
PATH = '/content/drive/My Drive/' + "SII_comportement_vol_ST"
# Chemin du dossier contenant les données: 
DATA_PATH = PATH + "/data/Dataset_V1_HDF5"
os.chdir(DATA_PATH) # on se place dans le dossier contenant les données 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# libraries 
import numpy as np
import pandas as pd
import pickle 
import re

In [8]:
# Retrieve the dataset 
df = pd.read_pickle("13_ST.pkl") 
print(df.shape)

(13, 4980)


## Identification of quantitative & qualitative variables.

In [9]:
# Importation du fichier contenant la description des paramètres
dfParams = pd.read_excel("parametres_xplane.xlsx")

# On ne conserve que les variables dont id != NaN
dfParams.dropna(subset = ["id"], axis = 0, inplace = True)

In [11]:
# Affichage de toutes les unités existantes avec leur type associé
#---------------------------------------------
units_list = dfParams.Units.unique() # liste de toutes les unités existantes
print("Nombre d'unités différentes: ", len(units_list))

# dataframe réduite à l'unité et à son type
dfUnits = dfParams[["Type","Units"]].drop_duplicates()
print("\nPour chaque unité, on affiche les types trouvés: ")
print("Unité, types:\n")

for u in units_list:
  types_u = list(dfUnits["Type"].iloc[np.where(dfUnits.Units == "boolean")]) # liste des types pour l'unité u affichés en liste
  print("{}: {}".format(u,types_u))

Nombre d'unités différentes:  54

Pour chaque unité, on affiche les types trouvés: 
Unité, types:

seconds: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
???: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
boolean: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
amps: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
degrees: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
degrees_magnetic: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
enum: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
bitfield: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
[0..1.5]: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
[0..1]: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
ratio: ['int', 'float', 'int[8]', 'int[2]', 'int[9]', 'int[10]', 'int[64]']
[-1..1]: ['int', 'float', 'int[8]', 'int[2]',

Toutes les variables ayant ces unités sont considérées **quantitatives**: 

*seconds, degrees, amps, degrees_magnetic, meters, percent, rad/sec, NewtonMeters, feet, mtr/sec, mtr/sec2, meters/sec, kias, NM, deg/sec, deg/sec2, quaternion, meters/second, fpm, kgs, kts, knots/mach, Knots, feet/minute, knots, meter, meter/s, feet/min, degreesC, degreesF, any, ftmsl, degc_or_f, EPR, kg/s, psi, degt, knots_mach, inHg, degm, ratio*

Explications: 
- amps: ampères
- kias: ? mais quantitatif
- NM: angular momentum
- quaternion: nombre dans un sens généralisé
- fpm: feet per second
- kgs: kilogrammes
- kts: vitesse en noeuds
- knots/mach: noeuds ou mach (vitesse du son) 
- any: pressure or temperature 
- ftmsl: altitude 
- degc_or_f: ? mais quantitatif
- EPR: engine pressure ratio
- psi: oil pressure
- knots_mach: variable associée toujours constante et = 200. Vitesse.
- inHg: variable accociée toujours constante et = 19.92. Baromètre
- degm: variable associée a toutes les valeurs sont 0. Mais je dirais quantitatif car la description ressemble à degt qui est quantitatif. A vérifier si on plus de données.
- ratio: variables quantitative pouvant prendre des valeurs entre 0 et 1.

Toutes les variables ayant ces unités sont considérées **qualitatives**: 

*boolean, bool, enum, bitfield, [0..1], [-1..1], [-0.5..1], 0..1, string, int, flags, [0..1.5]*

Explications: 
- boolean, bool: binaire, valeurs 0 ou 1
- enum, bitfield: plusieurs modalités par ex: 0,1,2,3 
- [0..1]: rangée de valeurs discrete: 0 ou 1 (VERIF)
- [-1..1] (valeurs +1,1), [-0.5..1] (valeurs -0.5,0,1), 0..1 (valeurs 0,1): rangées de valeurs discretes 
- string: flight id or ICAO code 
- int: prends valeurs 0,1
- flags: variable associée toujours constante et = 132. Autopilot mode
- [0..1.5]: variables associée a les valeurs sont 0. Mais je dirais qualitatif. A vérifier si on plus de données. 

Les variables dont l'unité vaut '???' seront traitées à la main plus bas. 


In [12]:
# Liste des unités associées à des variables QUANTITATIVES:
units_quanti = ["seconds","amps", "degrees","degrees_magnetic","ratio", "meters", "percent", "rad/sec", "NewtonMeters", 
"feet", "mtr/sec", "mtr/sec2", "meters/sec", "kias", "NM", "deg/sec", "deg/sec2", "quaternion", "meters/second", "fpm", 
"kgs", "kts", "knots/mach", "Knots", "feet/minute", "inHg", "knots", "meter", "meter/s", "feet/min", "degreesC", "degreesF", "any", 
"knots_mach", "ftmsl","degt", "degm", "degc_or_f", "EPR", "kg/s", "psi", ]

# Liste des unités associées à des variables QUALITATIVES:
units_quali = ["boolean", "bool", "enum", "bitfield", "[0..1]", "[-1..1]","[-0.5..1]", "0..1", "string", "int", "flags", "[0..1.5]"]

In [13]:
def flatten(l):
  """
  Transforme une liste de liste en liste: 
   [[1, 2, 3], [4, 5, 6], [7], [8, 9]] devient [1, 2, 3, 4, 5, 6, 7, 8, 9]
  """
  return [item for sublist in l for item in sublist]

In [14]:
# Récupération des variables associées aux unités QUANTITATIVES: 
VAR_QUANTI = []
for var in units_quanti:
  VAR_QUANTI.append(dfParams.Name.iloc[np.where(dfParams.Units == var)])
VAR_QUANTI = flatten(VAR_QUANTI)

# Récupération des variables associées aux unités QUALITATIVES: 
VAR_QUALI = []
for var in units_quali:
  VAR_QUALI.append(dfParams.Name.iloc[np.where(dfParams.Units == var)])
VAR_QUALI = flatten(VAR_QUALI)

print("Nombre de variables quantitatives (sans compter les doublons avec les teableaux): ", len(VAR_QUANTI))
print("Nombre de variables qualitatives  (sans compter les doublons avec les teableaux): ", len(VAR_QUALI))

Nombre de variables quantitatives (sans compter les doublons avec les teableaux):  568
Nombre de variables qualitatives  (sans compter les doublons avec les teableaux):  257


In [15]:
# On regarde plus attentivement les variables dont l'unité est "???"
#-----------------------------------------------
# variables associées à l'unité '???'
var_inconnues = dfParams.Name.iloc[np.where(dfParams.Units == "???")].values 
print("Il y a {} variables dont l'unité est '???' dans le fichier parameters_xplane.".format(len(var_inconnues)))
print("Nous allons assigner à la main ces variables au type qualitatif ou quantitatif.")

# Pour chaque variable dont l'utnité vaut '???', on tente de trouver son type à la main:
# Comme la description de certaines variables n'est pas explicite, le classement ci-dessous n'est pas certain. 
#-----------------------------------------------
# variables '???' présentes dans notre dataset qui sont qualitatives 
var_inconnues_quali = ["sim/cockpit/engine/fuel_tank_transfer",
                       "sim/cockpit/engine/fuel_tank_transfer_from",
                       "sim/cockpit/engine/fuel_tank_selector"] 
# variables '???' présentes dans notre dataset qui sont quantitatives  
var_inconnues_quanti = list(set(var_inconnues).difference(var_inconnues_quali)) 


# Ajout des variables inconnues à notre liste de variables: 
#-----------------------------------------------
VAR_QUANTI = VAR_QUANTI + var_inconnues_quanti
VAR_QUALI  = VAR_QUALI  + var_inconnues_quali

print("Nombre de variables quantitatives après ajout de '???' (sans compter les doublons avec les teableaux): ", len(VAR_QUANTI))
print("Nombre de variables qualitatives  après ajout de '???' (sans compter les doublons avec les teableaux): ", len(VAR_QUALI))
print("Vérification que les deux listes sont bien disjointes: intersection = ", list(set(VAR_QUALI).intersection(VAR_QUANTI)))

Il y a 21 variables dont l'unité est '???' dans le fichier parameters_xplane.
Nous allons assigner à la main ces variables au type qualitatif ou quantitatif.
Nombre de variables quantitatives après ajout de '???' (sans compter les doublons avec les teableaux):  586
Nombre de variables qualitatives  après ajout de '???' (sans compter les doublons avec les teableaux):  260
Vérification que les deux listes sont bien disjointes: intersection =  []


Nous venons de récupérer le nom de toutes les variables quantitatives et qualitatives. Cependant, certaines de ces variables sont des tableaux dont chaque colonne compte comme une variable à part entière. Il faut donc modifier nos listes de variables pour qu'elles prennent en compte le nom des variables correspondant à des tableaux. 

le type 'float[8][10]' n'est présent qu'une seule fois dans parameters_xplane et correspond à une seule variable: "sim/flightmodel/engine/vortex_ring_state". En regardant cette variable de plus près, on constate qu'il s'agit d'un tableau de taille [8] et donc le type est sans doute une erreur de saisie. On ne lit que le 1er entier dans le code ci-dessous, donc le code ne prendra en compte que le n°8. 


In [ ]:
print("Tous les types possibles pour les variables:\n", dfParams.Type.unique())

Tous les types possibles pour les variables:
 ['float' 'int' 'int[8]' 'int[2]' 'int[9]' 'float[56]' 'float[8]' 'double'
 'float[4]' 'byte[512]' 'float[64]' 'int[64]' 'float[20]' 'float[10]'
 'float[8][10]' 'int[10]']


In [16]:
# Mise à jour de la liste des variables quantitatives et qualitatives
# en prennant en compte les tableaux.
#-------------------------------------------------
def computeTabVarList(VAR, exept_var):
  """
  Cette fonction prend en argument la liste de toutes les variables qualitatives ou quantitatives 
  du dataset (sans compter les doublons avec les teableaux). La fonction renvoie la liste de toutes les 
  variables du dataset en ayant créé des "doublons" pour les tableaux en se basant sur le type des variables:
  ex: si "param" est de te type 'int[4]' alors "param" est un tableau de 4 entiers et donc la liste contenant 
  toutes les variables contiendra: "param0", "param1", "param2", "param3".
  
  exept_var : liste de variables dont on sait (à postériori) qu'elles vont créer 
  des erreurs et que l'on décide de traiter séparément. 
  """
  # liste contenant toutes les variables  (avec une variable pour chaque colonne des tableaux)
  VAR_TAB = []

  # on retire de VAR, les variables générant des erreurs (si elles sont présentes dans VAR)
  VAR = list(set(VAR).difference(exept_var))

  # on parcours les varibles  
  for v in VAR:
    # récupération du type de la variable v
    type_v = dfParams.Type.iloc[np.where(dfParams.Name == v)[0][0]]

    # on recherche le type de la variable v contient un nombre (ex: 'float[20]')
    nb = [int(s) for s in re.findall(r'\d+', type_v)]

    # si il n'y a pas de nombre
    if len(nb) == 0: 
      # on ajoute la variable v telle quelle dans la liste des variables 
      VAR_TAB += [v]

    # si il y a un nombre alors v est un tableau
    else: 
      # on ajoute à v le suffixe correspondant à l'indice du tableau: v devient v0, v1, v2...
      var_tab = []
      for i in range(nb[0]): 
        var_tab.append(v + str(i))
      # on ajoute le tableau créé dans la liste des variables 
      VAR_TAB += var_tab

  return VAR_TAB    

In [17]:
exept_var = ["sim/cockpit2/tcas/targets/position/flap_ratio", "sim/cockpit2/tcas/targets/position/flap_ratio2"]
# Mise à jour de la liste des variables quantitatives en prennant en compte les tableaux:
VAR_QUANTI_TAB = computeTabVarList(VAR_QUANTI,exept_var)
# Mise à jour de la liste des variables qualitatives en prennant en compte les tableaux:
VAR_QUALI_TAB = computeTabVarList(VAR_QUALI,exept_var)

In [18]:
# Ajout à la main des variables exept_var
# -----------------------------------------------------
l = "sim/cockpit2/tcas/targets/position/flap_ratio"
exept_var_tab = []

for v in df.columns:
  if v[:len(l)] == l:
    exept_var_tab += [v]

# Ajout des variables exept_var à la liste de toutes les variables
VAR_QUANTI_TAB += exept_var_tab # toutes ces variables sont quantitatives

In [28]:
# Traitement particulier de la variable 'sim/flightmodel/controls/elv_trim'
# qui a été assignée aux variables qualitatives alors que c'est une variable 
# quantitative.
VAR_QUANTI_TAB = VAR_QUANTI_TAB + ['sim/flightmodel/controls/elv_trim']
VAR_QUALI_TAB.remove('sim/flightmodel/controls/elv_trim')

In [29]:
print("Nombre de variables quantitatives trouvées: ", len(VAR_QUANTI_TAB))
print("Nombre de variables qualitatives trouvées: ", len(VAR_QUALI_TAB))
print("Vérification que les deux listes sont bien disjointes: Insersection = ", 
      list(set(VAR_QUALI_TAB).intersection(VAR_QUANTI_TAB)))
VAR_TOTAL_TAB = VAR_QUANTI_TAB + VAR_QUALI_TAB
print("Nombre total de variables trouvées (quantitatives + qualitatives): ", len(VAR_TOTAL_TAB))
print("Nombre total de variables dans notre dataset: ", len(df.columns))

# liste des variables qui sont dans le dataset mais pas dans parameters_xplane: 
var_diff = list(set(df.columns).difference(VAR_TOTAL_TAB)) 

print("\n==> Il y a {} variables dans le dataset qui ne sont pas dans l'excel 'parameters_xplane'.".format(len(var_diff)))

print("\nVérification que nous avons bien sélectionné toutes les variables: ")
print(np.all (np.sort(VAR_TOTAL_TAB) == np.sort(df.columns)))

Nombre de variables quantitatives trouvées:  3166
Nombre de variables qualitatives trouvées:  1814
Vérification que les deux listes sont bien disjointes: Insersection =  []
Nombre total de variables trouvées (quantitatives + qualitatives):  4980
Nombre total de variables dans notre dataset:  4980

==> Il y a 0 variables dans le dataset qui ne sont pas dans l'excel 'parameters_xplane'.

Vérification que nous avons bien sélectionné toutes les variables: 
True


In [ ]:
# Sauvegarde du nom des variables pour utilisation ultérieure
# -----------------------------------------------------
VAR_QUANTI_TAB = pd.Series(VAR_QUANTI_TAB)
VAR_QUALI_TAB  = pd.Series(VAR_QUALI_TAB)

VAR_QUALI_TAB.to_pickle("VAR_QUALI_TAB.pkl")
VAR_QUANTI_TAB.to_pickle("VAR_QUANTI_TAB.pkl")